## Preprocessing

In [1]:
import pandas as pd
df = pd.read_parquet('../data/dataset.parquet', engine='pyarrow')

Remove **time_to_failure** and **session_counter** and set **Timestamp** as the index.

In [2]:
df.drop(columns=['session_counter', 'time_to_failure'], inplace=True)
df.set_index('Timestamp', inplace=True)

## Modelling

In [3]:
import numpy as np

label = np.array(['alert_11'])
features = np.array(df.columns.difference(label))

print(f"-> Label:\t{label.shape}")
print(f"\t{label}")
print(f"-> Features:\t{features.shape}")
print(f"\t{features}")

-> Label:	(1,)
	['alert_11']
-> Features:	(13,)
	['Current speed cart [%]' 'Flag roping' 'Lifting motor speed [RPM]'
 'Lifting speed rotation [M/MIN]' 'Platform Motor frequency [HZ]'
 'Platform Position [°]' 'Platform motor speed [%]'
 'Platform rotation speed [RPM]' 'Slave rotation speed [M/MIN]'
 'Temperature hoist drive [°C]' 'Temperature platform drive [°C]'
 'Temperature slave drive [°C]' 'Tensione totale film [%]']


In [4]:
X = df[features].to_numpy()
y = df[label].to_numpy().flatten()

print(f"-> X:\t{X.shape}")
print(f"-> y:\t{y.shape}")

-> X:	(679045, 13)
-> y:	(679045,)


## MTS Sliding Window

In [5]:
import numpy as np 

def window(X_data, y_data, width: int, shift: int):
    
    X_wins, y_wins = [], []

    for index, (X, y) in enumerate(zip(X_data, y_data)):
        if (index + width + shift) <= X_data.shape[0]:

            window = slice((index + width), (index + width + shift))

            X_wins.append(X_data[index: index + width])

            y_values_shift = y_data[window]
            y_wins.append(int(np.any(y_values_shift == 1)))

    X_wins = np.array(X_wins)
    y_wins = np.array(y_wins)
    return X_wins.reshape(X_wins.shape[0], -1), y_wins.flatten()

In [6]:
X_wins, y_wins = window(X, y, width=120, shift=180)

In [7]:
print(f"-> X:\t{X_wins.shape}")
print(f"-> y:\t{y_wins.shape}")

-> X:	(678746, 1560)
-> y:	(678746,)


## Undersampling

In [8]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)

X_res, y_res = rus.fit_resample(X_wins, y_wins)

In [9]:
classes, distribution = np.unique(y_res, return_counts=True)

In [10]:
print(f"-> X_res:\t{X_res.shape}")
print(f"-> y_res:\t{y_res.shape}")

-> X_res:	(6648, 1560)
-> y_res:	(6648,)


In [11]:
print(f"-> {classes[0]}:\t{distribution[0]}")
print(f"-> {classes[1]}:\t{distribution[1]}")

-> 0:	3324
-> 1:	3324


## Models

Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=1234)

SVM (Support Vector Machine).

In [13]:
from sklearn.svm import SVC

svc = SVC(random_state=1234, C=1)

## K-Fold Cross Validation

In [31]:
from sklearn.model_selection import BaseCrossValidator
from sklearn.metrics import accuracy_score

class FixedSizeKFold(BaseCrossValidator):
    def __init__(self, n_splits : int = 5):
        self.n_splits = n_splits
    
    def get_n_splits(self, X = None, y = None, groups = None) -> int:
        return self.n_splits

    def split(self, X, y,  groups = None, val_size: float = 0.2):
        fold_size = len(X) // self.n_splits

        for fold in range(self.n_splits):
            start = fold * fold_size
            
            if fold == self.n_splits - 1:
                end = len(X)          
            else: 
                end = (fold + 1) * fold_size

            train_start = start
            train_end = int((1 - val_size) * fold_size)

            val_start = train_end
            val_end = end
            
            yield np.arange(train_start, train_end), np.arange(val_start, val_end)

In [32]:
cv = FixedSizeKFold(n_splits = 5)
for train_index, test_index in cv.split(X_res, y_res):
    print(train_index)
    print(test_index)

[   0    1    2 ... 1060 1061 1062]
[1063 1064 1065 1066 1067 1068 1069 1070 1071 1072 1073 1074 1075 1076
 1077 1078 1079 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 1090
 1091 1092 1093 1094 1095 1096 1097 1098 1099 1100 1101 1102 1103 1104
 1105 1106 1107 1108 1109 1110 1111 1112 1113 1114 1115 1116 1117 1118
 1119 1120 1121 1122 1123 1124 1125 1126 1127 1128 1129 1130 1131 1132
 1133 1134 1135 1136 1137 1138 1139 1140 1141 1142 1143 1144 1145 1146
 1147 1148 1149 1150 1151 1152 1153 1154 1155 1156 1157 1158 1159 1160
 1161 1162 1163 1164 1165 1166 1167 1168 1169 1170 1171 1172 1173 1174
 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1185 1186 1187 1188
 1189 1190 1191 1192 1193 1194 1195 1196 1197 1198 1199 1200 1201 1202
 1203 1204 1205 1206 1207 1208 1209 1210 1211 1212 1213 1214 1215 1216
 1217 1218 1219 1220 1221 1222 1223 1224 1225 1226 1227 1228 1229 1230
 1231 1232 1233 1234 1235 1236 1237 1238 1239 1240 1241 1242 1243 1244
 1245 1246 1247 1248 1249 1250 1251 1252 

In [33]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

skf = StratifiedKFold(n_splits=5)

scores = cross_val_score(rf, X_res, y_res, cv=skf)
print(f"RF:\t{scores.mean()}")

scores = cross_val_score(svc, X_res, y_res, cv=skf)
print(f"SVC:\t{scores.mean()}")

# scores = cross_val_score(lstm, X_res, y_res, cv=skf)
# print(f"LSTM:\t{scores.mean()}")

RF:	0.5670844153272572
SVC:	0.5914213298483229
